In [2]:
import argparse
import os
import datetime
import logging
import time

import torch
import torch.nn as nn
import torch.utils
import torch.distributed
from torch.utils.data import DataLoader
import multiprocessing

import numpy as np

from core.configs import cfg
from core.datasets import build_dataset
from core.models import build_feature_extractor, build_classifier
from core.solver import adjust_learning_rate
from core.utils.misc import mkdir
from core.utils.logger import setup_logger
from core.utils.metric_logger import MetricLogger
from core.active.build import PixelSelection, RegionSelection
from core.datasets.dataset_path_catalog import DatasetCatalog
from core.loss.negative_learning_loss import NegativeLearningLoss
from core.loss.local_consistent_loss import LocalConsistentLoss
from core.utils.utils import set_random_seed
from torch.utils.tensorboard import SummaryWriter

import warnings
warnings.filterwarnings('ignore')

In [3]:
os.system("nvidia-smi")

Thu Oct 13 03:10:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:C3:00.0 Off |                    0 |
| N/A   36C    P0    60W / 250W |    471MiB / 40960MiB |     99%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

0

In [1]:
torch.cuda.is_available()

NameError: name 'torch' is not defined

In [4]:
parser = argparse.ArgumentParser(description="Active Domain Adaptive Semantic Segmentation Training")
parser.add_argument("-cfg",
                    "--config-file",
                    default="",
                    metavar="FILE",
                    help="path to config file",
                    type=str)
parser.add_argument("--proctitle",
                    type=str,
                    default="RCL-AAA",
                    help="allow a process to change its title",)
parser.add_argument(
    "opts",
    help="Modify config options using the command-line",
    default=None,
    nargs=argparse.REMAINDER
)

# args = parser.parse_args()

args = parser.parse_args(args=['-cfg', 'configs/gtav/deeplabv3plus_r101_RA.yaml','--proctitle', 'RCL-AAA'])
args.opts = ['OUTPUT_DIR', 'results/v3plus_gtav_ra_5.0_precent', 'DEBUG', '0']

if args.opts is not None:
    args.opts[-1] = args.opts[-1].strip('\r\n')

torch.backends.cudnn.benchmark = True

cfg.merge_from_file(args.config_file)
cfg.merge_from_list(args.opts)
cfg.freeze()

output_dir = cfg.OUTPUT_DIR
if output_dir:
    mkdir(output_dir)

logger = setup_logger("RCL-AAA", output_dir, 0)
logger.info(args)

logger.info("Loaded configuration file {}".format(args.config_file))
logger.info("Running with config:\n{}".format(cfg))

logger.info('Initializing Cityscapes label mask...')

set_random_seed(cfg.SEED)

2022-10-05 20:25:40,712 RCL-AAA INFO: Namespace(config_file='configs/gtav/deeplabv3plus_r101_RA.yaml', opts=['OUTPUT_DIR', 'results/v3plus_gtav_ra_5.0_precent', 'DEBUG', '0'], proctitle='RCL-AAA')
2022-10-05 20:25:40,725 RCL-AAA INFO: Loaded configuration file configs/gtav/deeplabv3plus_r101_RA.yaml
2022-10-05 20:25:40,726 RCL-AAA INFO: Running with config:
ACTIVE:
  NAME: RCL-AAA
  PIXELS: 40
  RADIUS_K: 1
  RATIO: 0.05
  SELECT_ITER: [10000, 12000, 14000, 16000, 18000]
  SETTING: RA
DATASETS:
  SOURCE_TRAIN: gtav_train
  TARGET_TRAIN: cityscapes_train
  TEST: cityscapes_val
DEBUG: 0
INPUT:
  IGNORE_LABEL: 255
  INPUT_SCALES_TRAIN: (1.0, 1.0)
  INPUT_SIZE_TEST: (1280, 640)
  PIXEL_MEAN: [0.485, 0.456, 0.406]
  PIXEL_STD: [0.229, 0.224, 0.225]
  SOURCE_INPUT_SIZE_TRAIN: (1280, 720)
  TARGET_INPUT_SIZE_TRAIN: (1280, 640)
  TO_BGR255: False
MODEL:
  DEVICE: cuda
  FREEZE_BN: True
  NAME: deeplabv3plus_resnet101
  NUM_CLASSES: 19
  WEIGHTS: https://download.pytorch.org/models/resnet101-5d

In [5]:
print("Here is {} CPU, {} GPU.".format(multiprocessing.cpu_count(), torch.cuda.device_count()))
logger = logging.getLogger("RCL-AAA.trainer")
# tb_writer = SummaryWriter('./{}_{}_tensorboard_log_{}_confidence'.format(cfg.DATASETS.SOURCE_TRAIN.split('_')[0], cfg.DATASETS.TARGET_TRAIN.split('_')[0], cfg.CONFIDENCE.WEIGHT))
# print('Tensorboard writer log has been created at {}'.format('./{}_{}_tensorboard_log_{}_confidence'.format(cfg.DATASETS.SOURCE_TRAIN.split('_')[0], cfg.DATASETS.TARGET_TRAIN.split('_')[0], cfg.CONFIDENCE.WEIGHT)))

# create network
device = torch.device(cfg.MODEL.DEVICE)
feature_extractor = build_feature_extractor(cfg)
#feature_extractor = nn.DataParallel(feature_extractor)
feature_extractor.to(device)

classifier = build_classifier(cfg)
#classifier = nn.DataParallel(classifier)
classifier.to(device)
print()

Here is 64 CPU, 0 GPU.
load checkpoint from http path: https://download.pytorch.org/models/resnet101-5d3b4d8f.pth


RuntimeError: No CUDA GPUs are available

In [ ]:
# init optimizer
optimizer_fea = torch.optim.SGD(feature_extractor.parameters(), lr=cfg.SOLVER.BASE_LR, momentum=cfg.SOLVER.MOMENTUM,
                                weight_decay=cfg.SOLVER.WEIGHT_DECAY)
optimizer_fea.zero_grad()

optimizer_cls = torch.optim.SGD(classifier.parameters(), lr=cfg.SOLVER.BASE_LR * 10, momentum=cfg.SOLVER.MOMENTUM,
                                weight_decay=cfg.SOLVER.WEIGHT_DECAY)
optimizer_cls.zero_grad()

iteration = 0

In [ ]:
# # load checkpoint
# if cfg.resume:
#     logger.info("Loading checkpoint from {}".format(cfg.resume))
#     checkpoint = torch.load(cfg.OUTPUT_DIR + '/' + cfg.resume, map_location=torch.device('cpu'))
#     iteration = checkpoint['iteration']
#     feature_extractor.load_state_dict(checkpoint['feature_extractor'])
#     optimizer_fea.load_state_dict(checkpoint['optimizer_fea'])
#     classifier.load_state_dict(checkpoint['classifier'])
#     optimizer_cls.load_state_dict(checkpoint['optimizer_cls'])
# # feature_extractor = nn.DataParallel(feature_extractor)      # modified by CZC
# # classifier = nn.DataParallel(classifier)            # modified by CZC

In [ ]:
# init mask for cityscape
# DatasetCatalog.initMask(cfg)

In [ ]:
src_train_data = build_dataset(cfg, mode='train', is_source=True)
tgt_train_data = build_dataset(cfg, mode='train', is_source=False)
tgt_epoch_data = build_dataset(cfg, mode='active', is_source=False, epochwise=True)

src_train_loader = DataLoader(src_train_data, batch_size=2, shuffle=True, num_workers=4,
                              pin_memory=False, drop_last=True)
tgt_train_loader = DataLoader(tgt_train_data, batch_size=2, shuffle=True, num_workers=4,
                              pin_memory=False, drop_last=True)
tgt_epoch_loader = DataLoader(tgt_epoch_data, batch_size=1, shuffle=False, num_workers=4,
                              pin_memory=False, drop_last=True)

In [ ]:
torch.set_printoptions(threshold=np.inf)    # added by czc
# src_train_data[0]

In [ ]:
print(src_train_data[0]['img'].size())
print(src_train_data[0]['label'].size())

In [ ]:
src_input, src_label = src_train_data[0]['img'], src_train_data[0]['label']
src_input = src_input.cuda(non_blocking=True)
src_label = src_label.cuda(non_blocking=True)

In [ ]:
# init loss
sup_criterion = nn.CrossEntropyLoss(ignore_index=255)
# negative_criterion = NegativeLearningLoss(threshold=cfg.SOLVER.NEGATIVE_THRESHOLD)
# local_consistent_loss = LocalConsistentLoss(cfg.MODEL.NUM_CLASSES, cfg.SOLVER.LCR_TYPE).cuda()


start_warmup_time = time.time()
end = time.time()
max_iters = cfg.SOLVER.MAX_ITER
warmup_iters = 10000
meters = MetricLogger(delimiter="  ")

logger.info(">>>>>>>>>>>>>>>> Start Training >>>>>>>>>>>>>>>>")
feature_extractor.train()
classifier.train()
active_round = 1

In [ ]:
src_size = src_input.shape[-2:]
print(src_size)
src_input = src_input.unsqueeze(0)

In [ ]:
src_input.size()

In [ ]:
src_out = classifier(feature_extractor(src_input), size=src_size)
print(src_out.size())
print(src_out[0,:,0,0])

In [ ]:
predict = torch.softmax(src_out, dim=1)
print(predict[0,:,100,1000])
print(predict.size())

In [ ]:
# src_label

## Warm Up

In [ ]:
iteration=0
for batch_index, (src_data, tgt_data) in enumerate(zip(src_train_loader, tgt_train_loader)):       
    data_time = time.time() - end

    current_lr = adjust_learning_rate(cfg.SOLVER.LR_METHOD, cfg.SOLVER.BASE_LR, iteration, max_iters,
                                      power=cfg.SOLVER.LR_POWER)
    # tb_writer.add_scalar(tag="lr", scalar_value=current_lr, global_step=iteration)      # added by czc
    for index in range(len(optimizer_fea.param_groups)):
        optimizer_fea.param_groups[index]['lr'] = current_lr
    for index in range(len(optimizer_cls.param_groups)):
        optimizer_cls.param_groups[index]['lr'] = current_lr * 10

    optimizer_fea.zero_grad()
    optimizer_cls.zero_grad()

    src_input, src_label = src_data['img'], src_data['label']
    src_input = src_input.cuda(non_blocking=True)
    src_label = src_label.cuda(non_blocking=True)
    print(src_input.size())
    print(src_label.size())
    print(src_label.size()[0])


    src_size = src_input.shape[-2:]
    print(src_size)
    src_out = classifier(feature_extractor(src_input), size=src_size)
    # print(src_out)

    predict = torch.softmax(src_out, dim=1)
    print(predict.size())

    iteration += 1
    if iteration == 1:
        break

In [ ]:
from collections import Counter
h = 720
w = 1280
numparts_h = 2
numparts_w = 4
parts_h = int(h / numparts_h)
parts_w = int(w / numparts_w)
batch_size = 2
batch_centroids = {}        
# batch_centroids:{
#                  'img_idx': {
#                               {}_{}': {
#                                         classID: {centroid}, classID: {centroid}, classID: {centroid}, ...

import time
start = time.perf_counter()


for k in range(batch_size):
    batch_centroids['img_idx_{}'.format(k)] = {}
    for i in range(numparts_h):
        for j in range(numparts_w):
            batch_centroids['img_idx_{}'.format(k)]['{}_{}'.format(i,j)] = {} 
            
            # Get region coordinates 
            if [i,j] == [range(numparts_h)[-1], range(numparts_w)[-1]]:
                region_index = [i*parts_h, h-1, j*parts_w, w-1]
            else:
                region_index = [i*parts_h, (i+1)*parts_h-1, j*parts_w, (j+1)*parts_w-1]
            # batch_centroids['img_idx_{}'.format(k)]['region_{}_{}'.format(i,j)]['region_index'] = region_index
            
            # Get all class ID in a single region
            classID = dict(Counter(src_label[k, region_index[0]:region_index[1], region_index[2]:region_index[3]].cpu().numpy().flatten()))
            if classID.__contains__(255): del classID[255]
            # batch_centroids['img_idx_{}'.format(k)]['region_{}_{}'.format(i,j)]['region_class']['classID'] = classID
            
            # Get all predict mean as centroids
            centroids = {}
            for key in classID:
                predict_sum = torch.zeros([1,19], requires_grad=True)
                predict_sum = predict_sum.cuda(non_blocking=True)
                mask = (src_label[k, region_index[0]:region_index[1], region_index[2]:region_index[3]] == key)
                predict_mask = predict[k, :, region_index[0]:region_index[1], region_index[2]:region_index[3]] * mask
                centroids[key] = predict_mask.sum(axis=[1,2]) / classID[key]


            batch_centroids['img_idx_{}'.format(k)]['{}_{}'.format(i,j)] = centroids
            # print(batch_centroids)
            # print([i,j])
            
end = time.perf_counter()
print(str(end-start))

In [ ]:
print(batch_centroids)

In [ ]:
# intra contrastive loss v1
num_classes = 19
start = time.perf_counter()

for k in batch_centroids:
    loss = []
    # calculate per clas
    for cls in range(num_classes):
        pos, neg = {}, {}
        # pos: {'region_{}_{}': {tensor([...])} }
        # neg: {'region_{}_{}': {cls: tensor([...]), cls: tensor([...]), cls: tensor([...]), ...}}
        for region in batch_centroids[k]:  # 此时的region是 0_0
            neg[region] = {}
            for intra_cls in batch_centroids[k][region]:
                neg[region][intra_cls] = batch_centroids[k][region][intra_cls]    # 只有这样copy tensor，才不至于改变batch_centroids本身
            if batch_centroids[k][region].__contains__(cls):
                pos[region] = batch_centroids[k][region][cls]
                del neg[region][cls]
        print('class_{}:'.format(cls))
        print('pos:{}'.format(pos))
        # print('neg:{}'.format(neg))
end = time.perf_counter()
print(str(end-start))

In [ ]:
# intra contrastive loss v2
num_classes = 19
positive_weight_increment_step = 0.01
negative_weight_increment_step = 0.01

start = time.perf_counter()

for k in batch_centroids:
    loss = []
    # calculate per clas
    for cls in range(num_classes):
        pos, neg = {}, {}
        # pos: {'region_{}_{}': {tensor([...])} }
        # neg: {'region_{}_{}': {cls: tensor([...]), cls: tensor([...]), cls: tensor([...]), ...}}
        for region in batch_centroids[k]:  # 此时的region是 0_0
            neg[region] = {}
            for intra_cls in batch_centroids[k][region]:
                neg[region][intra_cls] = batch_centroids[k][region][intra_cls]    # 只有这样copy tensor，才不至于改变batch_centroids本身
            if batch_centroids[k][region].__contains__(cls):
                pos[region] = batch_centroids[k][region][cls]
                del neg[region][cls]
        pos_pairs = []
        for pos_region_1 in pos:
            pos_pairs.append(pos[pos_region_1]) # The first positive pair.
            for pos_region_2 in pos:
                if pos_region_2 != pos_region_1:    # 与其它区域的positive centroids构成带权重的positive pairs
                    pos_region_1_index = np.array([int(pos_region_1.split('_')[0]), int(pos_region_1.split('_')[1])])
                    pos_region_2_index = np.array([int(pos_region_2.split('_')[0]), int(pos_region_2.split('_')[1])])
                    positive_weight = 1 - positive_weight_increment_step * np.linalg.norm(pos_region_1_index - pos_region_2_index, ord=2) # L2 norm
                    pos_pairs.append(pos[pos_region_2] * positive_weight)
                else:
                    continue
            print(pos_pairs) # 该class，以region为中心所构建的所有pos_pairs
            pos_pairs_cl = torch.stack(pos_pairs, dim=0)    # 拼接所有positive pairs

        neg_pairs = []
        # for neg_region_1 in neg:
            
                    
end = time.perf_counter()
print(str(end-start))

In [94]:
def contrastive_loss(pos_set, neg_set, temperature):
    assert pos_set.size() != 0, "Positive pairs should not be EMPTY!"
    assert neg_set.size() != 0, "Negative pairs should not be EMPTY!"

    pos_head = torch.index_select(pos_set, 0, torch.tensor([0]))
    pos_pairs = torch.mm(pos_head, pos_set.permute(1,0))
    neg_pairs = torch.mm(pos_head, neg_set.permute(1,0))

    all_pairs = torch.cat([neg_pairs.repeat(pos_pairs.size()[1],1), pos_pairs.permute(1,0)], dim=1)
    all_pairs = torch.exp(all_pairs / temperature)

    exp_aggregation_row = all_pairs.sum(dim=1, keepdim=True)
    frac_row = torch.index_select(all_pairs, 1, torch.tensor([all_pairs.size()[1] - 1])) / exp_aggregation_row
    log_row = torch.log(frac_row)
    
    if pos_set.size()[0] == 1:
        cl_loss = log_row * (-1)
    else:
        cl_loss = torch.mean(log_row[1:,:]) * (-1)
    
    return cl_loss


In [97]:
loss = []
for i in range(1,5):
    cat = torch.tensor([[0.1,0.3,0.6],[0.4,0.5,0.1]])
    dog = torch.tensor([[0.7,0.1,0.2],[0.3,0.3,0.4],[0.2,0.3,0.5]])
    loss.append(contrastive_loss(cat, dog, 0.05))
    print(loss)

[tensor(3.5584)]
[tensor(3.5584), tensor(3.5584)]
[tensor(3.5584), tensor(3.5584), tensor(3.5584)]
[tensor(3.5584), tensor(3.5584), tensor(3.5584), tensor(3.5584)]


In [61]:
import torch
a = torch.tensor([1,2,3,4,5,6], dtype=torch.float)
print("a:", a)
b = torch.tensor([9,10,11]).unsqueeze(dim=0)
print("b:", b)
c = a.repeat(b.size()[1],1)
print("c:", c)
print("b.T:", b.permute(1,0))
d = torch.cat((c,b.permute(1,0)), dim=1)
print("d:", d)
a/torch.tensor([[1],[2],[3]])

a: tensor([1., 2., 3., 4., 5., 6.])
b: tensor([[ 9, 10, 11]])
c: tensor([[1., 2., 3., 4., 5., 6.],
        [1., 2., 3., 4., 5., 6.],
        [1., 2., 3., 4., 5., 6.]])
b.T: tensor([[ 9],
        [10],
        [11]])
d: tensor([[ 1.,  2.,  3.,  4.,  5.,  6.,  9.],
        [ 1.,  2.,  3.,  4.,  5.,  6., 10.],
        [ 1.,  2.,  3.,  4.,  5.,  6., 11.]])


tensor([[1.0000, 2.0000, 3.0000, 4.0000, 5.0000, 6.0000],
        [0.5000, 1.0000, 1.5000, 2.0000, 2.5000, 3.0000],
        [0.3333, 0.6667, 1.0000, 1.3333, 1.6667, 2.0000]])

In [ ]:
# intra contrastive loss v3
num_classes = 19
positive_weight_increment_step = 0.01
negative_weight_increment_step = 0.01
temperature = 0.07

start = time.perf_counter()
loss = []

for k in batch_centroids:
    # calculate per clas
    for cls in range(num_classes):
        pos, neg = {}, {}
        # pos: {'region_{}_{}': {tensor([...])} }
        # neg: {'region_{}_{}': {cls: tensor([...]), cls: tensor([...]), cls: tensor([...]), ...}}
        for region in batch_centroids[k]:  # 此时的region是 0_0
            neg[region] = {}
            for intra_cls in batch_centroids[k][region]:
                neg[region][intra_cls] = batch_centroids[k][region][intra_cls]    # 只有这样copy tensor，才不至于改变batch_centroids本身
            if batch_centroids[k][region].__contains__(cls):
                pos[region] = batch_centroids[k][region][cls]
                del neg[region][cls]

        pos_region = pos.keys() # positive pairs所在的region
        neg_region = neg.keys() # negative pairs所在的region
        all_region = list(set(pos_region + neg_region)) # 合并两个list，并且删除重复元素
        for region_1 in all_region:
            pos_per_region, neg_per_region = [], []         # 该类以该region为中心的positive和negative pairs
            cl_per_region = []
            region_1_index = np.array([int(region_1.split('_')[0]), int(region_1.split('_')[1])])
            if pos.__contains__(region_1):
                pos_per_region.append(pos[region_1])        # positive pairs的头，即cls在region_1的centroids
            for neg_cls_1 in neg[region_1]:
                neg_per_region.append(neg[region_1][neg_cls_1])   # negatiave pairs的头，即在region_1中除了cls之外的所有centroids
            for region_2 in all_region:                         # 收集其它region的positive pairs和negative pairs
                if region_2 != region_1:
                    region_2_index = np.array([int(region_2.split('_')[0]), int(region_2.split('_')[1])])
                    positive_weight = 1 + positive_weight_increment_step * np.linalg.norm(region_1_index - region_2_index, ord=2) # L2 norm
                    negative_weight = 1 - negative_weight_increment_step * np.linalg.norm(region_1_index - region_2_index, ord=2) # L2 norm
                    if pos.__contains__(region_2):     # 其它区域的positive pairs，将会乘上权重
                        pos_per_region.append(pos[region_2] * positive_weight)
                    if neg.__contains__(region_2):     # 其它区域的negative pairs，将会乘上权重
                        for neg_cls_2 in neg[region_2]:
                            neg_per_region.append(neg[region_2][neg_cls_2] * negative_weight)
            pos_set_cl = torch.stack(pos_per_region, dim=0)       # 第一行tensor就是用于query的positive pair头，剩下都是所有乘上权重后的positive pairs
            neg_set_cl = torch.stack(neg_per_region, dim=0)       # 所有乘上权重后的negative pairs
            
            cl_per_region = contrastive_loss(pos_set_cl, neg_set_cl, temperature)

            loss.append([cl_per_region])


        
        # for neg_region_1 in neg:
            
print(loss)              
end = time.perf_counter()
print(str(end-start))

In [ ]:
pos_region_1 = '0_1'
pos_region_2 = '3_5'
pos_region_1_index = np.array([int(pos_region_1.split('_')[0]), int(pos_region_1.split('_')[1])])
pos_region_2_index = np.array([int(pos_region_2.split('_')[0]), int(pos_region_2.split('_')[1])])
print(pos_region_1_index)
print(pos_region_2_index)
print(pos_region_1_index - pos_region_2_index)
np.linalg.norm(pos_region_1_index - pos_region_2_index, ord=2)

[0 1]
[3 5]
[-3 -4]


5.0